# 線形代数ライブラリの実装についてメモ

- numpyやscipyの内部ではLAPACK（C/C++の線形代数ライブラリ）を使っている
- LAPACKはだいたいLU分解をしている

### 逆行列は連立一次方程式の問題に帰着できる

逆行列$A^{-1}$を求めたい場合

$$
A X = B
$$

という連立一次方程式を、$B=I$として

$$
A X = I
$$

とすることで$X=A^{-1}$を得ることができる


### LAPACKで連立一次方程式を解く

正則な係数行列をもつ連立一次方程式をLAPACKで解くには、xGESV関数（連立一次方程式を解く関数）もしくはxGETRF関数（LU分解の関数）とxGETRS関数（前進代入・後退代入を行う関数）を使えば解ける（幸谷, 2016）

xGESV関数は$Ax=b$を$x$について解き、$b:=x$として$b$に代入する関数

## 逆行列計算の実装を覗く

### scipy.linalg.inv

- docs: [scipy.linalg.inv — SciPy v1.12.0 Manual](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.inv.html)
- source: [source](https://github.com/scipy/scipy/blob/v1.12.0/scipy/linalg/_basic.py#L903-L976)

scipyのinvはGETRF、GETRI

In [5]:
# scipyのinv
import numpy as np
from scipy import linalg
A = np.array([
    [1, 2],
    [3, 4]
])
linalg.inv(A)

array([[-2. ,  1. ],
       [ 1.5, -0.5]])

### numpy.linalg.inv

- docs: [numpy.linalg.inv — NumPy v1.26 Manual](https://numpy.org/doc/stable/reference/generated/numpy.linalg.inv.html)
- source:
    - [numpy/numpy/linalg/linalg.py](https://github.com/numpy/numpy/blob/v1.26.0/numpy/linalg/linalg.py#L492-L562)
    - [numpy/numpy/linalg/umath_linalg.cpp](https://github.com/numpy/numpy/blob/v1.26.0/numpy/linalg/umath_linalg.cpp#L1811-L1843)
        - gesv関数を呼び出している

In [14]:
import numpy as np
np.linalg.inv(A)

array([[-2. ,  1. ],
       [ 1.5, -0.5]])

## scipyのLAPACK functions

[Low-level LAPACK functions (scipy.linalg.lapack) — SciPy v1.12.0 Manual](https://docs.scipy.org/doc/scipy/reference/linalg.lapack.html)

#### xGESV

- [scipy.linalg.lapack.dgesv — SciPy v1.12.0 Manual](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lapack.dgesv.html#scipy.linalg.lapack.dgesv)
- [LAPACK: dgesv](https://netlib.org/lapack/explore-html-3.6.1/d7/d3b/group__double_g_esolve_ga5ee879032a8365897c3ba91e3dc8d512.html)

> DGESV computes the solution to a real system of linear equations
>
> A * X = B,
>
> where A is an N-by-N matrix and X and B are N-by-NRHS matrices.
>
> The LU decomposition with partial pivoting and row interchanges is used to factor A as 
>
> A = P * L * U,
>
> where P is a permutation matrix, L is unit lower triangular, and U is upper triangular.
> The factored form of A is then used to solve the system of equations A * X = B.

In [9]:
from scipy.linalg.lapack import dgesv

# AX = I としてX=A^{-1}を推定する
I = np.eye(A.shape[0])

lu, piv, x, info = dgesv(A, I)

In [10]:
x

array([[-2. ,  1. ],
       [ 1.5, -0.5]])

#### xGETRF

LU分解をする関数。

[scipy.linalg.lapack.dgetrf — SciPy v1.12.0 Manual](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lapack.dgetrf.html#scipy.linalg.lapack.dgetrf)

docs: [LAPACK: dgetrf](https://www.netlib.org/lapack/explore-html-3.6.1/dd/d9a/group__double_g_ecomputational_ga0019443faea08275ca60a734d0593e60.html)

> DGETRF computes an LU factorization of a general M-by-N matrix A using partial pivoting with row interchanges.
> The factorization has the form
>
>    A = P * L * U
>
> where P is a permutation matrix, L is lower triangular with unit diagonal elements (lower trapezoidal if m > n), and U is upper triangular (upper trapezoidal if m < n).


In [20]:
from scipy.linalg.lapack import dgetrf

lu, piv, info = dgetrf(A)

In [21]:
lu

array([[3.        , 4.        ],
       [0.33333333, 0.66666667]])

In [22]:
piv

array([1, 1], dtype=int32)

#### xGETRI

LU分解の結果から逆行列を取得する関数

[scipy.linalg.lapack.dgetri — SciPy v1.12.0 Manual](https://docs.scipy.org/doc/scipy/reference/generated/scipy.linalg.lapack.dgetri.html#scipy.linalg.lapack.dgetri)

cpp docs: [LAPACK: dgetri](https://www.netlib.org/lapack/explore-html-3.6.1/dd/d9a/group__double_g_ecomputational_ga56d9c860ce4ce42ded7f914fdb0683ff.html)

>  DGETRI computes the inverse of a matrix using the LU factorization computed by DGETRF.
>
> This method inverts U and then computes inv(A) by solving the system inv(A)*L = inv(U) for inv(A).

In [23]:
from scipy.linalg.lapack import dgetri

inv_a, info = dgetri(lu, piv)
inv_a

array([[-2. ,  1. ],
       [ 1.5, -0.5]])

## LAPACKはどうやって計算量を下げているのか

掃き出し法とLU分解は計算量的に大差ないらしい（[安易に逆行列を数値計算するのはやめよう](https://opqrstuvcut.github.io/blog/posts/%E5%AE%89%E6%98%93%E3%81%AB%E9%80%86%E8%A1%8C%E5%88%97%E3%82%92%E6%95%B0%E5%80%A4%E8%A8%88%E7%AE%97%E3%81%99%E3%82%8B%E3%81%AE%E3%81%AF%E3%82%84%E3%82%81%E3%82%88%E3%81%86/)
）

## 逆行列の計算量

- [数値解析第12回逆行列と計算量](https://www.st.nanzan-u.ac.jp/info/sugiurah/NumericalAnalysis/files/Notes/NANoteA12.pdf)
- [安易に逆行列を数値計算するのはやめよう](https://opqrstuvcut.github.io/blog/posts/%E5%AE%89%E6%98%93%E3%81%AB%E9%80%86%E8%A1%8C%E5%88%97%E3%82%92%E6%95%B0%E5%80%A4%E8%A8%88%E7%AE%97%E3%81%99%E3%82%8B%E3%81%AE%E3%81%AF%E3%82%84%E3%82%81%E3%82%88%E3%81%86/)


## 参考

- 幸谷智紀. (2016). LAPACK/BLAS 入門: Linear Algebra PACKage Basic Linear Algebra Subprograms.
- [渡部 善隆「連立1次方程式の直接解法とソフトウェア」](https://ri2t.kyushu-u.ac.jp/~watanabe/RESERCH/MANUSCRIPT/KOHO/WSMP/main.pdf)
